<a href="https://colab.research.google.com/github/Yashopant/Natural_Laungage_Inference_using_RoberTa/blob/master/Natural_Laungage_Inference_using_RoberTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
from transformers import BertTokenizer, TFRobertaModel

In [3]:
from google.colab import files


uploaded = files.upload()

Saving NLP.csv to NLP.csv


In [4]:
import pandas as pd
import io

train_df = pd.read_csv(io.BytesIO(uploaded['NLP.csv']))

In [5]:
train_df.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [6]:
train_df['language'].value_counts()

English       6870
Chinese        411
Arabic         401
French         390
Swahili        385
Urdu           381
Vietnamese     379
Russian        376
Hindi          374
Greek          372
Thai           371
Spanish        366
Turkish        351
German         351
Bulgarian      342
Name: language, dtype: int64

In [7]:
train_df['label'].value_counts()

0    4176
2    4064
1    3880
Name: label, dtype: int64

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [9]:
def encoded_sentence(text):
    encoded_input = tokenizer(text)
    return encoded_input

In [10]:
eg_output = encoded_sentence("I want to be a scientist")
eg_output

{'input_ids': [101, 146, 21528, 10114, 10347, 169, 59248, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
def inputidstypes_sentence(premise,hypothesis):
    output = encoded_sentence(premise)
    input_ids = output['input_ids']
    input_types = output['token_type_ids']
    hypothesis_tokens = list(tokenizer.tokenize(hypothesis))
    hypothesis_tokens.append('[SEP]')
    hypothesis_ids = tokenizer.convert_tokens_to_ids(hypothesis_tokens)
    input_types.extend([1] * len(hypothesis_tokens))
    input_ids.extend(hypothesis_ids)
    return input_ids,input_types

In [12]:
def model_input(premises,hypotheses):

    num_examples = len(hypotheses)
    print(num_examples)

    input_ids = list()
    input_types = list()

    for premise,hypothesis in zip(premises,hypotheses):
        input_id,input_type = inputidstypes_sentence(premise,hypothesis)
        input_ids.append(input_id)
        input_types.append(input_type)

    print(len(input_ids))
    print(len(input_types))
    input_words_ids =  tf.ragged.constant(input_ids)
    input_types_ids = tf.ragged.constant(input_types)
    input_mask = tf.ones_like(input_words_ids).to_tensor(shape=[num_examples, 259])
    print(input_words_ids.get_shape())

    inputs = {
      'input_word_ids': input_words_ids.to_tensor(shape=[num_examples, 259]),
      'input_mask': input_mask,
      'input_type_ids': input_types_ids.to_tensor(shape=[num_examples, 259])}

    return inputs


In [13]:
xtrain = model_input(train_df.premise,train_df.hypothesis)


12120
12120
12120
(12120, None)


In [14]:
max_len = 259

def build_model():
    bert_encoder = TFRobertaModel.from_pretrained("bert-base-multilingual-cased")
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")

    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5),loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model

In [15]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

Number of replicas: 1


In [16]:
with strategy.scope():
    model = build_model()
    model.summary()

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['bert.encoder.layer.2.attention.self.key.bias', 'bert.encoder.layer.3.attention.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.output.dense.weight', 'bert.encoder.layer.9.attention.self.query.weight', 'bert.encoder.layer.4.output.LayerNorm.weight', 'bert.encoder.layer.10.attention.output.dense.bias', 'bert.encoder.layer.4.attention.self.key.bias', 'bert.encoder.layer.8.intermediate.dense.bias', 'bert.encoder.layer.5.attention.output.LayerNorm.weight', 'bert.encoder.layer.5.attention.self.query.bias', 'bert.encoder.layer.5.attention.self.key.bias', 'bert.encoder.layer.10.attention.self.query.bias', 'bert.encoder.layer.11.intermediate.dense.bias', 'bert.encoder.layer.4.attention.self.query.bias', 'bert.encoder.layer.10.output.dense.bias', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.11.output.LayerNor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_word_ids (InputLayer  [(None, 259)]                0         []                            
 )                                                                                                
                                                                                                  
 input_mask (InputLayer)     [(None, 259)]                0         []                            
                                                                                                  
 input_type_ids (InputLayer  [(None, 259)]                0         []                            
 )                                                                                                
                                                                                              

In [29]:
model.fit(xtrain, train_df.label.values, epochs = 10, verbose = 1, batch_size = 16, validation_split = 0.2)

Epoch 1/10


606/606 [==============================] - 579s 897ms/step - loss: 1.3557 - accuracy: 0.3264 - val_loss: 1.1682 - val_accuracy: 0.3111
Epoch 2/10
606/606 [==============================] - 589s 972ms/step - loss: 1.1336 - accuracy: 0.3381 - val_loss: 1.1300 - val_accuracy: 0.3449
Epoch 3/10
606/606 [==============================] - 546s 901ms/step - loss: 1.1205 - accuracy: 0.3365 - val_loss: 1.1472 - val_accuracy: 0.3111
Epoch 4/10
606/606 [==============================] - 589s 971ms/step - loss: 1.1113 - accuracy: 0.3385 - val_loss: 1.1182 - val_accuracy: 0.3111
Epoch 5/10
606/606 [==============================] - 585s 966ms/step - loss: 1.1071 - accuracy: 0.3323 - val_loss: 1.1249 - val_accuracy: 0.3111
Epoch 6/10
606/606 [==============================] - 586s 966ms/step - loss: 1.1042 - accuracy: 0.3330 - val_loss: 1.1027 - val_accuracy: 0.3111
Epoch 7/10
606/606 [==============================] - 586s 968ms/step - loss: 1.1019 - accuracy: 0.3373 - val_loss: 1.1021 - val_accura